# Using PyTorch-based models in GluonTS

This notebook illustrates how one can implement a time series model using PyTorch, and use it together with the rest of the GluonTS ecosystem for data loading, feature processing, and model evaluation.

In [ ]:
from typing import List, Optional, Callable
from itertools import islice

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [ ]:
from gluonts.dataset.field_names import FieldName
from gluonts.dataset.loader import TrainDataLoader
from gluonts.dataset.repository.datasets import get_dataset
from gluonts.evaluation import Evaluator
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.torch.batchify import batchify 
from gluonts.torch.support.util import copy_parameters
from gluonts.torch.model.predictor import PyTorchPredictor
from gluonts.transform import Chain, AddObservedValuesIndicator, InstanceSplitter, ExpectedNumInstanceSampler

For this example we will use the "electricity" dataset, which can be loaded as follows.

In [ ]:
dataset = get_dataset("electricity")

This is what the first time series from the training portion of the dataset look like:

In [ ]:
date_formater = mdates.DateFormatter('%Y')

fig = plt.figure(figsize=(12,8))
for idx, entry in enumerate(islice(dataset.train, 9)):
    ax = plt.subplot(3, 3, idx+1)
    t = pd.date_range(start=entry["start"], periods=len(entry["target"]), freq=entry["start"].freq)
    plt.plot(t, entry["target"])
    plt.xticks(pd.date_range(start=pd.to_datetime("2011-12-31"), periods=3, freq="AS"))
    ax.xaxis.set_major_formatter(date_formater)


## Probabilistic feed-forward network using PyTorch

We will use a pretty simple model, based on a feed-forward network whose output layer produces the parameters of a Student's t-distribution at each time step in the prediction range. We will define two networks based on this idea:
* The `TrainingFeedForwardNetwork` computes the loss associated with given observations, i.e. the negative log-likelihood of the observations according to the output distribution; this will be used during training.
* The `SamplingFeedForwardNetwork` will be used at inference time: this uses the output distribution to draw a sample of a given size, as a way to encode the predicted distribution.

In [ ]:
def mean_abs_scaling(context, min_scale=1e-5):
    return context.abs().mean(1).clamp(min_scale, None).unsqueeze(1)

In [ ]:
def no_scaling(context):
    return torch.ones(context.shape[0], 1)

In [ ]:
class TrainingFeedForwardNetwork(nn.Module):
    distr_type = torch.distributions.StudentT
    
    def __init__(
        self,
        prediction_length: int,
        context_length: int,
        hidden_dimensions: List[int],
        batch_norm: bool=False,
        scaling: Callable=mean_abs_scaling,
    ) -> None:
        super().__init__()
        
        assert prediction_length > 0
        assert context_length > 0
        assert len(hidden_dimensions) > 0
        
        self.prediction_length = prediction_length
        self.context_length = context_length
        self.hidden_dimensions = hidden_dimensions
        self.batch_norm = batch_norm
        self.scaling = scaling
        
        dimensions = [context_length] + hidden_dimensions[:-1]

        modules = []
        for in_size, out_size in zip(dimensions[:-1], dimensions[1:]):
            modules += [self.__make_lin(in_size, out_size), nn.ReLU()]
            if batch_norm:
                modules.append(nn.BatchNorm1d(units))
        modules.append(self.__make_lin(dimensions[-1], prediction_length * hidden_dimensions[-1]))
        self.nn = nn.Sequential(*modules)
        
        self.df_proj = nn.Sequential(self.__make_lin(hidden_dimensions[-1], 1), nn.Softplus())
        self.loc_proj = self.__make_lin(hidden_dimensions[-1], 1)
        self.scale_proj = nn.Sequential(self.__make_lin(hidden_dimensions[-1], 1), nn.Softplus())
    
    @staticmethod
    def __make_lin(dim_in, dim_out):
        lin = nn.Linear(dim_in, dim_out)
        torch.nn.init.uniform_(lin.weight, -0.07, 0.07)
        torch.nn.init.zeros_(lin.bias)
        return lin
    
    def distr_and_scale(self, context):
        scale = self.scaling(context)
        scaled_context = context / scale
        nn_out = self.nn(scaled_context)
        nn_out_reshaped = nn_out.reshape(-1, self.prediction_length, self.hidden_dimensions[-1])
        
        distr_args = (
            2.0 + self.df_proj(nn_out_reshaped).squeeze(dim=-1),
            self.loc_proj(nn_out_reshaped).squeeze(dim=-1),
            self.scale_proj(nn_out_reshaped).squeeze(dim=-1),
        )
        distr = net.distr_type(*distr_args)
        
        return distr, scale
    
    def forward(self, context, target):
        assert context.shape[-1] == self.context_length
        assert target.shape[-1] == self.prediction_length
        
        distr, scale = self.distr_and_scale(context)
        loss = (-distr.log_prob(target / scale) + torch.log(scale)).mean(dim=1)
        
        return loss

In [ ]:
class SamplingFeedForwardNetwork(TrainingFeedForwardNetwork):
    def __init__(self, *args, num_samples: int = 1000, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_samples = num_samples
        
    def forward(self, context):
        assert context.shape[-1] == self.context_length
        
        distr, scale = self.distr_and_scale(context)
        sample = distr.sample((self.num_samples, )) * scale
        
        return sample.permute(1, 0, 2)

We can now instantiate the training network, and explore its set of parameters.

In [ ]:
context_length = 2 * 7 * 24
prediction_length = dataset.metadata.prediction_length
hidden_dimensions = [96, 48]

In [ ]:
batch_size = 32
num_batches_per_epoch = 100

In [ ]:
net = TrainingFeedForwardNetwork(
    prediction_length=prediction_length,
    context_length=context_length,
    hidden_dimensions=hidden_dimensions,
    batch_norm=False,
    scaling=mean_abs_scaling,
)

In [ ]:
sum(np.prod(p.shape) for p in net.parameters())

In [ ]:
for p in net.parameters():
    print(p.shape)

## Defining the training data loader

We now set up the data loader which will yield batches of data to train on. Starting from the original dataset, the data loader is configured to apply the following transformation, which does essentially two things:
* Replaces `nan`s in the target field with a dummy value (zero), and adds a field indicating which values were actually observed vs imputed this way.
* Slices out training instances of a fixed length randomly from the given dataset; these will be stacked into batches by the data loader itself.

In [ ]:
transformation = Chain([
    AddObservedValuesIndicator(
        target_field=FieldName.TARGET,
        output_field=FieldName.OBSERVED_VALUES,
    ),
    InstanceSplitter(
        target_field=FieldName.TARGET,
        is_pad_field=FieldName.IS_PAD,
        start_field=FieldName.START,
        forecast_start_field=FieldName.FORECAST_START,
        train_sampler=ExpectedNumInstanceSampler(num_instances=1),
        past_length=context_length,
        future_length=prediction_length,
        time_series_fields=[FieldName.OBSERVED_VALUES],
    ),
])

In [ ]:
data_loader = TrainDataLoader(
    dataset.train,
    batch_size=batch_size,
    stack_fn=batchify,
    transform=transformation,
    num_batches_per_epoch=num_batches_per_epoch
)

## Train the model

We can now train the model using any of the available optimizers from PyTorch:

In [ ]:
optimizer = torch.optim.Adam(net.parameters())

for epoch_no in range(10):
    sum_epoch_loss = 0.0
    for batch_no, batch in enumerate(data_loader, start=1):
        optimizer.zero_grad()

        context = batch["past_target"]
        target = batch["future_target"]
        loss_vec = net(context, target)
        loss = loss_vec.mean()
        loss.backward()
        
        optimizer.step()
        
        sum_epoch_loss += loss.detach().numpy().item()
        
    print(f"{epoch_no}: {sum_epoch_loss / num_batches_per_epoch}")

## Create predictor out of the trained model, and test it

We now have a trained model, whose parameters can be copied over to a `SamplingFeedForwardNetwork` object: we will wrap this into a `PyTorchPredictor` that can be used for inference tasks.

In [ ]:
pred_net = SamplingFeedForwardNetwork(
    prediction_length=net.prediction_length,
    context_length=net.context_length,
    hidden_dimensions=net.hidden_dimensions,
    batch_norm=net.batch_norm,
)
copy_parameters(net, pred_net)

predictor_pytorch = PyTorchPredictor(
    prediction_length=prediction_length, freq = dataset.metadata.freq, 
    input_names = ["past_target"], prediction_net=pred_net, batch_size=32, input_transform=transformation,
    device=None
)

For example, we can do backtesting on the test dataset: in what follows, `make_evaluation_predictions` will slice out the trailing `prediction_length` observations from the test time series, and use the given predictor to obtain forecasts for the same time range.

In [ ]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=dataset.test,
    predictor=predictor_pytorch,
    num_samples=1000,
)

forecasts_pytorch = list(forecast_it)
tss_pytorch = list(ts_it)

Once we have the forecasts, we can plot them:

In [ ]:
plt.figure(figsize=(20, 15))
date_formater = mdates.DateFormatter('%b, %d')
plt.rcParams.update({'font.size': 15})

for idx, (forecast, ts) in islice(enumerate(zip(forecasts_pytorch, tss_pytorch)), 9):
    ax =plt.subplot(3, 3, idx+1)
    
    plt.plot(ts[-5 * prediction_length:], label="target")
    forecast.plot()
    plt.xticks(rotation=60)
    ax.xaxis.set_major_formatter(date_formater)
    
plt.gcf().tight_layout()
plt.legend()
plt.show()

And we can compute evaluation metrics, that summarize the performance of the model on our test data.

In [ ]:
evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])

In [ ]:
metrics_pytorch, _ = evaluator(iter(tss_pytorch), iter(forecasts_pytorch), num_series=len(dataset.test))
pd.DataFrame.from_records(metrics_pytorch, index=["FeedForward"]).transpose()